In [ ]:
!pip install transformers


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset
from accelerate import Accelerator
import os

# Vérifier la disponibilité du GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("GPU is available and being used.")
else:
    device = torch.device('cpu')
    print("GPU is not available, using CPU.")

# Charger le tokenizer et le modèle BERT pour le MLM
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint).to(device)  # Allouer le modèle au GPU

# Charger le fichier texte
additional_texts = []
with open('/kaggle/input/ccccccc/concatenated_text_file (2).txt', 'r', encoding='utf-8') as file:  # Remplacez '/kaggle/input/votre_fichier.txt' par le chemin de votre fichier
    for line in file:
        additional_texts.append(line.strip())

# Convertir en Dataset
additional_dataset = Dataset.from_dict({"text": additional_texts})

# Tokeniser le dataset additionnel
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)  # Réduire max_length à 128

tokenized_additional_dataset = additional_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Préparer le collateur de données pour MLM
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# Définir le chemin de sauvegarde des checkpoints et des modèles


# Définir les arguments de formation avec Kaggle comme chemin de sauvegarde
training_args = TrainingArguments(
     output_dir="/kaggle/working/diseaseBert",
    overwrite_output_dir=True,
    num_train_epochs=40,  # Utiliser un grand nombre d'époques pour le pré-entraînement
    per_device_train_batch_size=8,  # Réduire la taille du lot
    save_steps=8000,  # Sauvegarder les checkpoints toutes les 500 étapes
    save_total_limit=2,
    prediction_loss_only=True,
    learning_rate=5e-5,  # Taux d'apprentissage utilisé dans BERT original
    weight_decay=0.01,  # Décroissance de poids utilisée dans BERT original
    fp16=True  # Utiliser le mode mixed precision
)

# Initialiser le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_additional_dataset,
)

# Entraîner le modèle
trainer.train()


!zip -r /kaggle/working/diseaseBert.zip /kaggle/working/diseaseBert

In [ ]:
pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub()